In [1]:
import PyKDL as kdl
%pylab inline
np.set_printoptions(precision=4,suppress=True)

Populating the interactive namespace from numpy and matplotlib


# Kinematic Calibration of the Universal Robots UR10 Manipulator

 ![](http://www.universal-robots.com/media/22419/ur_product_ur10.png)

## Denavit-Hartenberg Parameters

In [2]:
dh_a = [0.000, -0.612, -0.5723, 0.000000, 0.0000, 0.0000]
dh_d = [0.1273,  0.000,  0.0000, 0.163941, 0.1157, 0.0922]
dh_alpha = [ 1.570796327, 0, 0, 1.570796327, -1.570796327, 0 ]
q_home_offset = [0, -1.570796327, 0, -1.570796327, 0, 0]
joint_direction = [1, 1, -1, 1, 1, 1]
num_of_joints = 6

## Link Parameters

In [3]:
mass = [7.1, 12.7, 4.27, 2.000, 2.000, 0.365]
center_of_mass = [[0.021, 0.000, 0.027], 
                  [0.38, 0.000, 0.158], 
                  [0.24, 0.000, 0.068], 
                  [0.000, 0.007, 0.018], 
                  [0.000, 0.007, 0.018], 
                  [0, 0, -0.026] ]
inertia_matrix = np.array([[0.03408, 0.00002, -0.00425, 0.00002, 0.03529, 0.00008, 0.00425, 0.00008, 0.02156], 
                  [0.02814, 0.00005, -0.01561, 0.00005, 0.77068, 0.00002, -0.01561, 0.00002, 0.76943], 
                  [0.01014, 0.00008, 0.00916, 0.00008, 0.30928, 0.00000, 0.00916, 0.00000, 0.30646], 
                  [0.00296, -0.00001, -0.00000, -0.00001, 0.00222, -0.00024, -0.00000, -0.00024, 0.00258], 
                  [0.00296, -0.00001, -0.00000, -0.00001, 0.00222, -0.00024, -0.00000, -0.00024, 0.00258], 
                  [0.00040, 0.00000,  -0.00000, 0.00000, 0.00041, 0.00000, -0.00000, 0.00000, 0.00034]])
gravity = [0.0, 0.0, 9.82]

## Orocos 

In [4]:
chain = kdl.Chain()
for a, alpha, d in zip(dh_a, dh_alpha, dh_d):
    joint = kdl.Joint(kdl.Joint.RotZ)
    frame = kdl.Frame().DH(a, alpha, d, 0)
    chain.addSegment(kdl.Segment(joint, frame))

In [5]:
jnt_jac_solver = kdl.ChainJntToJacSolver(chain)
fk_solver_pos = kdl.ChainFkSolverPos_recursive(chain)
fk_solver_vel = kdl.ChainFkSolverVel_recursive(chain)
ik_solver_vel = kdl.ChainIkSolverVel_pinv(chain)
ik_solver_pos = kdl.ChainIkSolverPos_NR(chain, fk_solver_pos, ik_solver_vel)

In [6]:
q = kdl.JntArray(6)
# q[0] = 0.5
# q[1] = pi/2 
qdot = kdl.JntArray(6)
qdot[1] = -1.0
kdl_frame_vel = kdl.FrameVel()
kdl_joint_vel = kdl.JntArrayVel(6)
kdl_joint_vel.q = q
kdl_joint_vel.qdot = qdot
fk_solver_vel.JntToCart(kdl_joint_vel, kdl_frame_vel, -1)
twist = kdl_frame_vel.GetTwist()
print(twist)

[     -0.1157,-2.42904e-10,      1.1843,           0,           1, 2.05103e-10]


In [7]:
frame = kdl.Frame()
# q[1] = -pi/2
fk_solver_pos.JntToCart(q, frame)
np.array(frame.p)

array([     -1.1843,   -0.256141,      0.0116], dtype=object)

In [12]:
jac = kdl.Jacobian(6)
jnt_jac_solver.JntToJac(q,jac)
J = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        J[i,j] = jac[i,j]
print(J)

[[ 0.2561  0.1157  0.1157  0.1157 -0.0922  0.    ]
 [-1.1843  0.      0.      0.      0.      0.    ]
 [ 0.     -1.1843 -0.5723  0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.     -1.     -1.     -1.      0.     -1.    ]
 [ 1.     -0.     -0.     -0.     -1.     -0.    ]]


In [9]:
K = np.diag([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
K

array([[ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  2.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  3.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  6.]])

In [10]:
r = np.linalg.norm(np.array([-0.11, 0.16, 0.33]) - np.array([0.0, 0.16, 0.13]))
r * cos(-pi/4) * 9.81

1.5833352156129166